In [1]:
import json
from jiwer import wer
from pathlib import Path
from text_normalizer.persian_normalizer import persian_normalizer_no_punc, normalize_persian_halfspace
from nemo.collections.asr.models import EncDecHybridRNNTCTCBPEModel

In [2]:
import sys_append

from utils.evaluate import evaluate_asr

In [3]:
# asr_model = EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="nvidia/stt_fa_fastconformer_hybrid_large").eval()
asr_model = EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="hsekhalilian/Speech_To_Text_Finetuning_03").eval()

[NeMo I 2025-07-28 13:59:23 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-07-28 13:59:24 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /home/user/.cache/asr-finetuning/datasets/combined/train_manifest.json
    sample_rate: 16000
    batch_size: 16
    shuffle: true
    num_workers: 8
    pin_memory: true
    max_duration: 20
    min_duration: 0.1
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: fully_randomized
    bucketing_batch_size: null
    
[NeMo W 2025-07-28 13:59:24 nemo_logging:405] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /home/user/.cache/asr-finetuning/datasets/hsekhalilian___fleurs/manifest

[NeMo I 2025-07-28 13:59:24 nemo_logging:393] PADDING: 0
[NeMo I 2025-07-28 13:59:25 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-28 13:59:25 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-28 13:59:26 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-07-28 13:59:27 nemo_logging:393] Model EncDecHybridRNNTCTCBPEModel was successfully restored from /home/jovyan/.cache/huggingface/hub/models--hsekhalilian--Speech_To_Text_Finetuning_03/snapshots/e31d364053dd63723a829d8cbe8f4970b5f7d474/Speech_To_Text_Finetuning_03.nemo.


In [4]:
manifest_path = "/home/jovyan/.cache/asr-finetuning/datasets/hsekhalilian___fleurs/manifests/validation_manifest.json"
# manifest_path = "/home/jovyan/.cache/asr-finetuning/datasets/hsekhalilian___sorted_commonvoice/manifests/test_manifest.json"

In [5]:
audio_files = []
references = []
predictions = []

with open(manifest_path, "r", encoding="utf-8") as f:
    for line in f:
        entry = json.loads(line)
        audio_file = Path.home() / ".cache" / entry["audio_filepath"].split(".cache/")[-1]
        audio_files.append(str(audio_file))
        references.append(entry["text"])

In [6]:
results = asr_model.transcribe(audio_files, batch_size=16)
predictions = [result.text for result in results]

Transcribing: 100%|██████████| 24/24 [00:11<00:00,  2.07it/s]


In [7]:
error_rate = wer(references, predictions)
print(f"\nWER: {error_rate:.2%}")


WER: 31.52%


In [8]:
evaluate_asr(references, predictions)

{'wer': 0.21046174436761098, 'cer': 0.09710577424091706}

In [9]:
reference_normalized = [normalize_persian_halfspace(persian_normalizer_no_punc(ref)) for ref in references]
hypothesis_normalized  = [normalize_persian_halfspace(persian_normalizer_no_punc(pred)) for pred in predictions]

reference_normalized = [ref.replace("\u200c", " ") for ref in reference_normalized]
hypothesis_normalized  = [pred.replace("\u200c", " ") for pred in hypothesis_normalized ]

error_rate = wer(reference_normalized, hypothesis_normalized)
print(f"\nWER: {error_rate:.2%}")


WER: 21.05%


In [10]:
for reference, prediction in zip(references, predictions):
    print("="*30)
    print("reference: ", reference)
    print("predictio: ", prediction)

reference:  بزرگترین مسابقه سال در ماه دسامبر در زمین‌های چوگان لاس کانیتاس اتفاق می‌افتد
predictio:  بزرگترین مسابقه سال در ماه دسام در زمینهای چوگان لازکانیکاس اتفاق می افتد
reference:  قدرت همه جانبه آن همه را تحت تاثیر قرار داد از پاشاه گرفته تا افراد عادی
predictio:  قدرت همه جانبهی آن همه را تحت تاثیر قرار داد از پادشاه گرفته تا افراد عادی
reference:  از این رو هر گونه اصلاحیه یا تقاضایی باید از طریق متصدی آژانس مسافرتی و نه بطور مستقیم از طریق هتل صورت گیرد
predictio:  از این رو هرگونه اصلاحیه یا تقاضایی باید از طریق متصدی آژانس مسافرتی و نه بهطور مستقیم از طریق هتل صورت گیرد
reference:  ماسیج کرزل اسکی‌باز مرد لهستانی که معلولیت بینایی داشت بود و راهنمای او آنا اوگارزینسکا اسکی مارپیچ سرعت را در جایگاه سیزدهم به پایان رساندند پارک سئورک جونگ از کره‌جنوبی مسابقات اسکی مارپیچ سرعت نشسته مردان را با مقام بیست و چهارمی به پایان رساند
predictio:  ماسیچکر اسکی باز مرد لهستانی که معلولیت بین داشت بود و راهنمای او آناگارزینس ها اسکی مال پیچ و سرعت را در جایگاه سیزام به پایان رساندند پا